In [1]:
suppressPackageStartupMessages(library(tidyverse))

suppressPackageStartupMessages(source("../R/process_slim_output.R"))

Conflicts with tidy packages ---------------------------------------------------


In [4]:
sim_dir <- "../simulations/different_models"

In [6]:
get_nea_sfs <- function(sim_dir, model, h, gen) {
    # load the whole SLiM output file first
    slim_file <- read_slim_file(file.path(traject_dir,
                                          paste0(model,
                                                 "__h_", h,
                                                 "__init_nea_0.1__rep_1__dump_gen_",
                                                 gen,
                                                 ".txt")))

    # get the Ne of modern humans in this generation
    N <- filter(read_populations(slim_file), pop_id == "p3")$pop_size

    neutral_exonic <-
        read_mutations(slim_file, "m1", "p2") %>%
        mutate(model=model, kind="exonic", gen=gen, freq=freq / (2 * N))
    
    neutral_nonexonic <-
        read_mutations(slim_file, "m1", "p3") %>%
        mutate(model=model, kind="nonexonic", gen=gen, freq=freq / (2 * N))
    
    bind_rows(neutral_exonic, neutral_nonexonic)
}